Neste tópico abordaremos a seleção de features, utilizando o método `Botura` e teste de `Chi2`para tal. Vale salientar que não utilizaremos correlação de Pearson(ou Spearman), pois nosso target não é numérico nem categórico ordinal.

Chi2: Método utilizado para seleção de features categóricas ordinais e nominais. É um metódo que avalia a indepedência das features em relação ao target.

Boruta: Método muito robusto e baseado em modelo, que leva em consideração a interação entre a features, algo nao levado em conta pelo Chi2 ou outros métodos univariados de seleção. Para mais infomações sobre o método: <a href="https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a">Clique Aqui!</a>. 


#  Depêndencias

In [1]:
#Importando bibliotecas.
#import sys
#!{sys.executable} -m pip install boruta
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cloudpickle
import matplotlib.pyplot as plt
from boruta import BorutaPy
from src.selecao_de_feature import boruta_selector,chi_squared,variance_threshold_selector
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

# Dados e Constantes

In [2]:
#Constantes
DATA_RAW_PATH = os.path.join('..','data','raw')
DATA_RAW_NAME = 'Safra_2018-2019.csv'
DATA_INTER_PATH = os.path.join('..','data','inter')
DATA_INTER_TRAIN_NAME = 'train_inter.csv'
DATA_INTER_TEST_NAME = 'test_inter.csv'
DATA_INTER_TRAIN_FT_SELECTION = 'train_inter_ft_selection.csv'
IMG_PATH = os.path.abspath(os.path.join('..','img'))
META_DATA_PATH = os.path.join(DATA_RAW_PATH,'dicionario_tipos.pkl')
TARGET = 'dano_na_plantacao'

In [3]:
with open(META_DATA_PATH,'rb') as f:
    dicionario_tipo = cloudpickle.load(f)
df_train = pd.read_csv(os.path.join(DATA_INTER_PATH, DATA_INTER_TRAIN_NAME), dtype=dicionario_tipo)

In [4]:
df_train = pd.read_csv(os.path.join(DATA_INTER_PATH, DATA_INTER_TRAIN_NAME), dtype=dicionario_tipo)

In [5]:
df_train.head()

,Estimativa_de_Insetos,Tipo_de_Cultivo,Tipo_de_Solo,Categoria_Pesticida,Doses_Semana,Semanas_Utilizando,Semanas_Sem_Uso,Temporada,dano_na_plantacao
0,984,0,1,2,25,12.0,19,3,0
1,851,0,0,2,20,27.0,7,2,0
2,283,0,0,3,35,27.0,0,3,0
3,2542,0,0,3,15,50.0,0,2,2
4,850,0,1,2,30,14.0,24,2,0


# Colunas Variância aprox = 0

Aqui a ideia é remover todas as colunas numéricas cuja variância é baixa.

In [12]:
#Utilizando apenas as variaveis numericas.
todas_as_variaveis = set(df_train.columns.tolist())
variaveis_categoricas = set(df_train.select_dtypes(include=['object']).columns.tolist())
variaveis_numericas = todas_as_variaveis - variaveis_categoricas
variaveis_numericas = list(variaveis_numericas)

In [13]:
#DataFrame com os dados com variancia maior que 0,01.
df_train_variance_threshold = variance_threshold_selector(df_train.loc[:,variaveis_numericas], threshold=0.01)

In [14]:
df_train_categoricas = df_train.loc[:,variaveis_categoricas]

In [15]:
#Juntando os dados de variancia maior que 0,01 com outras features categoricas.
df_train = pd.concat([df_train_variance_threshold, df_train_categoricas],axis=1)

In [16]:
df_train.shape

(64000, 9)

In [17]:
#Trazendo as novas variaveis depois da conversão de categórica para ordinal.
todas_as_variaveis = set(df_train.columns.tolist())
variaveis_categoricas_nominal_e_ordinal = set(df_train.select_dtypes(include=['object']).columns.tolist())
variaveis_numericas = todas_as_variaveis - variaveis_categoricas_nominal_e_ordinal
variaveis_categoricas_nominal_e_ordinal = list(variaveis_categoricas)

In [18]:
df_train.loc[:,variaveis_categoricas_nominal_e_ordinal].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tipo_de_Solo         64000 non-null  object
 1   Tipo_de_Cultivo      64000 non-null  object
 2   Categoria_Pesticida  64000 non-null  object
 3   Temporada            64000 non-null  object
 4   dano_na_plantacao    64000 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


# Abordagem (Chi2+Boruta)

## Chi2

Será utilizado teste de Chi2 para verificar independências das variáveis categóricas nominais e ordinais 
em relação ao target.

In [19]:
#Aplicando Chi2
chi2_df, logs = chi_squared(df_train, y = 'dano_na_plantacao', cols = variaveis_categoricas_nominal_e_ordinal)

In [20]:
chi2_df

,column,p-value,chi2_value
0,Tipo_de_Solo,2.171827e-06,26.079883
1,Tipo_de_Cultivo,4.815159e-10,42.908164
2,Categoria_Pesticida,0.000000e+00,5715.303638
3,Temporada,7.086076e-02,8.636334
4,dano_na_plantacao,NaN,NaN


In [21]:
logs

['A coluna dano_na_plantacao não pode ser avaliada. ']

Como `dano_na_plantacao` é o próprio target, não calculamos o chi2 para ela.
(Hipótese nula - H0: As variáveis são independentes)
Segundo o teste de hipotése realizado, as variáveis cujo p-valor < 0.05 tem relação com o target.

In [22]:
#As variáveis que serão excluídas. 
lista_descart_chi2 = chi2_df[chi2_df['p-value']>=0.05]['column'].tolist()

In [23]:
lista_descart_chi2

['Temporada']

A seleção com Chi2 conseguiu descartar a feature `Temporada`. 

## Boruta

In [24]:
#Vamos filtrar as Features removidas no step anterior antes de passarmos par ao Boruta
list_to_drop_boruta = boruta_selector(df_train.drop(lista_descart_chi2,axis=1), y='dano_na_plantacao')

In [25]:
list_to_drop_boruta

['Tipo_de_Solo']

A seleção com boruta conseguiu descartar a feature `Tipo_de_Solo`.

In [26]:
#Lista de dropagem Geral (chi2 + boruta)
lista_drop_geral = set(lista_descart_chi2+list_to_drop_boruta)
lista_drop_geral

{'Temporada', 'Tipo_de_Solo'}

In [27]:
# dropando as colunas pela abordagem.
df_train.drop(lista_drop_geral,axis=1).to_csv(os.path.join(DATA_INTER_PATH,DATA_INTER_TRAIN_FT_SELECTION), index=False)

A abordagem para seleção de features que utilizamos são (Chi2 e Boruta), que combinados trazem resultado bacanas. As features descartadas foram:`Temporada` e `Tipo_de_Solo`. Ambas não serão consideradas para nosso modelo. 